# Chapter 4 — C3D Utilities: Mapping, Scaling & Quick Plots (Markers & GRFs)

> **This notebook is part of Chapter 4 of the eBook.**  
> It may need libraries from previous chapters.  
> **Author:** Hossein Mokhtarzadeh — **Poseiq.com**  
> Find our work on **Udemy**.

This notebook matches your 'two-cells' workflow and uses the fixed C3D path:

```
/content/Eb015pi.c3d
```
Please ensure this file exists (it should have been provided or produced in earlier chapters).

## Setup
If needed, (re)install packages used here:

```bash
pip install ezc3d matplotlib numpy
```

In [ ]:
pip install ezc3d matplotlib numpy

## Load the C3D (fixed path)

In [ ]:
from pathlib import Path
import ezc3d
import numpy as np
import matplotlib.pyplot as plt

c3d_path = Path('/content/Eb015pi.c3d')
assert c3d_path.exists(), (
    f"Couldn't find {c3d_path}. Please place the file there — you should have it from previous chapters."
)

c3d = ezc3d.c3d(str(c3d_path))
print('Loaded:', c3d_path)


## Cell 1 — Utilities (mapping + scaling + helpers)

In [ ]:
# --- C3D helpers: analog normalization, scaling, mapping, quick plots ---
import numpy as np, matplotlib.pyplot as plt

def _pval(P,g,k, default=None):
    try: return P[g][k]['value']
    except Exception: return default

def _normalize_analogs(A):
    A = np.asarray(A)
    if A.ndim == 2: return A
    if A.ndim == 3:
        # (subframes, ch, frames) -> moveaxis; else (ch, subframes, frames)
        if A.shape[0] < 16 and A.shape[1] > A.shape[0]:
            return np.moveaxis(A, 0, -1).reshape(A.shape[1], -1)
        return A.reshape(A.shape[0], -1)
    return A.reshape(A.shape[0], -1)

def _clean_labels(v):
    try:
        if isinstance(v,(list,tuple)) and v and isinstance(v[0],(list,tuple,np.ndarray)):
            v = [x[0] for x in v]
    except Exception:
        pass
    return [str(x) for x in list(v or [])]

def c3d_streams(c3d):
    P = c3d['parameters']
    pts = c3d['data']['points']
    n_points, n_frames = pts.shape[1], pts.shape[2]
    pt_rate = float(_pval(P,'POINT','RATE',[1.0])[0]) if _pval(P,'POINT','RATE',None) is not None else 1.0
    t_pts   = np.arange(n_frames) / (pt_rate if pt_rate>0 else 1.0)
    point_labels = _clean_labels(_pval(P,'POINT','LABELS', []) or [f'P{i+1}' for i in range(n_points)])
    point_units  = (_pval(P,'POINT','UNITS', [''])[0] if _pval(P,'POINT','UNITS', None) else '')

    raw_an = c3d['data'].get('analogs', None)
    an = _normalize_analogs(raw_an) if raw_an is not None else np.zeros((0,0))
    an_rate = float(_pval(P,'ANALOG','RATE',[1.0])[0]) if _pval(P,'ANALOG','RATE',None) is not None else 1.0
    t_an = np.arange(an.shape[1]) / (an_rate if an_rate>0 else 1.0)
    analog_labels = _clean_labels(_pval(P,'ANALOG','LABELS', []) or [f'CH{i+1}' for i in range(an.shape[0])])
    analog_units  = _clean_labels(_pval(P,'ANALOG','UNITS', []) or ['']*an.shape[0])

    return P, pts, t_pts, point_labels, point_units, an, t_an, analog_labels, analog_units, an_rate

def c3d_scalers(P, n_ch):
    gen_scale = float((_pval(P,'ANALOG','GEN_SCALE',[1.0]) or [1.0])[0])
    def _to1(x,N,fill):
        try: arr = np.array(x).astype(float).ravel()
        except Exception: arr = np.array([], float)
        if arr.size < N: arr = np.concatenate([arr, np.full(N-arr.size, fill)])
        if arr.size > N: arr = arr[:N]
        return arr
    offs = _to1(_pval(P,'ANALOG','OFFSET',[0.0]*n_ch), n_ch, 0.0)
    sca  = _to1(_pval(P,'ANALOG','SCALE', [1.0]*n_ch), n_ch, 1.0)
    def apply(y,i, use=True): return (y - offs[i]) * sca[i] * gen_scale if use else y
    return apply

def forceplate_mapping(P, analog_labels):
    used = int((_pval(P,'FORCE_PLATFORM','USED',[0]) or [0])[0])
    ch   = _pval(P,'FORCE_PLATFORM','CHANNEL', None)
    mat = None
    if used and ch is not None:
        A = np.array(ch)
        if A.ndim == 1 and A.size == 6*used: A = A.reshape(6,used)
        if A.ndim == 2 and A.shape == (used,6): A = A.T
        if A.ndim == 2 and A.shape[0] == 6: mat = A.astype(int) - 1  # to 0-based
    low = [s.lower() for s in analog_labels]
    fxL = [i for i,s in enumerate(low) if 'fx' in s]
    fyL = [i for i,s in enumerate(low) if 'fy' in s]
    fzL = [i for i,s in enumerate(low) if 'fz' in s]
    return used, mat, fxL, fyL, fzL

def ch_index(p, comp, mat, an_n, fxL, fyL, fzL):
    if mat is not None and 0 <= p < mat.shape[1]:
        row = {'Fx':0,'Fy':1,'Fz':2,'Mx':3,'My':4,'Mz':5}[comp]
        j = int(mat[row, p]);  return j if 0 <= j < an_n else None
    L = {'Fx':fxL, 'Fy':fyL, 'Fz':fzL}[comp]
    return L[p] if p < len(L) else None

def list_forceplate_channels(c3d):
    P, _, _, _, _, an, _, analog_labels, _, _ = c3d_streams(c3d)
    used, mat, fxL, fyL, fzL = forceplate_mapping(P, analog_labels)
    nP = (mat.shape[1] if mat is not None else min(len(fxL), len(fyL), len(fzL)))
    print(f'Detected plates: {nP}  (FORCE_PLATFORM/USED={used})')
    for p in range(nP):
        out = []
        for comp in ['Fx','Fy','Fz','Mx','My','Mz']:
            j = ch_index(p, comp, mat, an.shape[0], fxL, fyL, fzL)
            if j is None: out.append(f"{comp}: -")
            else: out.append(f"{comp}: {analog_labels[j]} (ch {j})")
        print(' Plate {p}: '.replace('{p}', str(p+1)) + ' | '.join(out))

def plot_marker(c3d, label, axis='Z'):
    P, pts, t_pts, point_labels, point_units, *_ = c3d_streams(c3d)
    if label not in point_labels:
        print('Marker not found. Example:', point_labels[:5]); return
    comp = {'X':0,'Y':1,'Z':2}[axis.upper()]
    idx  = point_labels.index(label)
    y    = pts[comp, idx, :]
    plt.figure(figsize=(9,3)); plt.plot(t_pts, y); plt.grid(True, alpha=.3)
    plt.xlabel('Time [s]'); plt.ylabel(f"{axis} {point_units}"); plt.title(f"{label} ({axis})"); plt.tight_layout(); plt.show()

def plot_grf_board(c3d, invert_fz=False, include_mag=True, same_ylim=False, scale=True):
    P, _, _, _, _, an, t_an, analog_labels, _, _ = c3d_streams(c3d)
    if an.size == 0: print('No analog channels.'); return
    apply = c3d_scalers(P, an.shape[0])
    used, mat, fxL, fyL, fzL = forceplate_mapping(P, analog_labels)
    nP = (mat.shape[1] if mat is not None else min(len(fxL), len(fyL), len(fzL)))
    if nP == 0: print('No Fx/Fy/Fz detected.'); return

    rows = nP
    fig, axes = plt.subplots(rows, 1, figsize=(10, 3*rows), sharex=True)
    if rows == 1: axes = [axes]
    gmin, gmax = +np.inf, -np.inf
    cache = []
    for p in range(nP):
        ix,iy,iz = ch_index(p,'Fx',mat,an.shape[0],fxL,fyL,fzL), ch_index(p,'Fy',mat,an.shape[0],fxL,fyL,fzL), ch_index(p,'Fz',mat,an.shape[0],fxL,fyL,fzL)
        if None in (ix,iy,iz): cache.append(None); continue
        Fx,Fy,Fz = an[ix].copy(), an[iy].copy(), an[iz].copy()
        Fx,Fy,Fz = apply(Fx,ix,scale), apply(Fy,iy,scale), apply(Fz,iz,scale)
        if invert_fz: Fz = -Fz
        entry = {'Fx':Fx,'Fy':Fy,'Fz':Fz}
        if include_mag: entry['|F|'] = np.sqrt(Fx**2 + Fy**2 + Fz**2)
        cache.append(entry)
        cat = np.hstack(list(entry.values()))
        gmin, gmax = min(gmin, np.nanmin(cat)), max(gmax, np.nanmax(cat))
    for p,ax in enumerate(axes):
        if cache[p] is None:
            ax.text(0.5,0.5,f'Plate {p+1}: channels not found', ha='center', va='center', transform=ax.transAxes)
            ax.set_axis_off(); continue
        for name,y in cache[p].items(): ax.plot(t_an,y,label=name,linewidth=1.2)
        ax.set_title(f'Plate {p+1}'); ax.set_ylabel('Force'); ax.grid(True,alpha=.3)
        if same_ylim and np.isfinite(gmin) and np.isfinite(gmax):
            pad = 0.05*(gmax-gmin + 1e-9); ax.set_ylim(gmin-pad, gmax+pad)
    axes[-1].set_xlabel('Time [s]'); axes[0].legend(loc='upper right',ncol=4,frameon=False,fontsize=9)
    plt.tight_layout(); plt.show()


## Cell 2 — Marker / GRF quickplot (no ipywidgets)

In [ ]:
#@title Marker / GRF quickplot (works without ipywidgets)
mode = 'GRF board' #@param ['GRF board','Force plate channel','Marker POINT']
# --- GRF board options ---
invert_fz = False  #@param {type:'boolean'}
include_mag = True  #@param {type:'boolean'}
same_ylim = False  #@param {type:'boolean'}
apply_scaling = True  #@param {type:'boolean'}
# --- Single force-plate channel options ---
plate_index = 0  #@param {type:'integer'}
component = 'Fz' #@param ['Fx','Fy','Fz','|F|','Mx','My','Mz']
# --- Marker options ---
marker_label = 'RASI' #@param {type:'string'}
marker_axis = 'Z'     #@param ['X','Y','Z']

# Auto extract streams
P, pts, t_pts, point_labels, point_units, an, t_an, analog_labels, analog_units, an_rate = c3d_streams(c3d)

if mode == 'GRF board':
    print('Detected plates / channels:')
    list_forceplate_channels(c3d)
    plot_grf_board(c3d, invert_fz=invert_fz, include_mag=include_mag, same_ylim=same_ylim, scale=apply_scaling)

elif mode == 'Force plate channel':
    used, mat, fxL, fyL, fzL = forceplate_mapping(P, analog_labels)
    nP = (mat.shape[1] if mat is not None else min(len(fxL), len(fyL), len(fzL)))
    if nP == 0:
        print('No Fx/Fy/Fz detected.');
    else:
        p = max(0, min(plate_index, nP-1))
        apply = c3d_scalers(P, an.shape[0])

        def pick(comp):
            j = ch_index(p, comp, mat, an.shape[0], fxL, fyL, fzL)
            return (None, None) if j is None else (an[j].copy(), j)

        if component == '|F|':
            Fx, ix = pick('Fx'); Fy, iy = pick('Fy'); Fz, iz = pick('Fz')
            if None in (Fx,Fy,Fz):
                print('Required Fx/Fy/Fz missing for this plate.')
            else:
                if apply_scaling: Fx, Fy, Fz = apply(Fx,ix,True), apply(Fy,iy,True), apply(Fz,iz,True)
                if invert_fz: Fz = -Fz
                y = np.sqrt(Fx**2 + Fy**2 + Fz**2); ylab = 'Force'
                title = f'Plate {p+1} |F|'
                plt.figure(figsize=(10,3)); plt.plot(t_an, y); plt.grid(True, alpha=.3)
                plt.xlabel('Time [s]'); plt.ylabel(ylab); plt.title(title + (' (scaled)' if apply_scaling else ''))
                plt.tight_layout(); plt.show()
        else:
            y, j = pick(component)
            if y is None:
                print(f"{component} missing for plate {p+1}")
            else:
                if apply_scaling: y = apply(y, j, True)
                if component == 'Fz' and invert_fz: y = -y
                ylab = (analog_units[j] if j < len(analog_units) else '')
                title = f'Plate {p+1} {component}'
                plt.figure(figsize=(10,3)); plt.plot(t_an, y); plt.grid(True, alpha=.3)
                plt.xlabel('Time [s]'); plt.ylabel(ylab); plt.title(title + (' (scaled)' if apply_scaling else ''))
                plt.tight_layout(); plt.show()

elif mode == 'Marker POINT':
    if marker_label not in point_labels:
        print('Marker not found. Example labels:', point_labels[:10])
    else:
        plot_marker(c3d, marker_label, marker_axis)


## How to use
1. Run **Load the C3D** cell (ensuring `/content/Eb015pi.c3d` exists).
2. Run **Cell 1 — Utilities** once.
3. In **Cell 2 — Quickplot**, choose a `mode` and any options, then run the cell. No ipywidgets required.


## Example: Minimal hands-on (uses the same fixed path)

In [ ]:
# Step 1: (Optional) install
# !pip install ezc3d
import numpy as np, matplotlib.pyplot as plt
import ezc3d
from pathlib import Path

# Step 2: Load the fixed C3D
c3d_path = Path('/content/Eb015pi.c3d')
assert c3d_path.exists(), 'Place Eb015pi.c3d at /content (from previous chapters).'
c3d_ex = ezc3d.c3d(str(c3d_path))

points = c3d_ex['data']['points']      # (4, n_markers, n_frames)
analogs = c3d_ex['data']['analogs']    # (n_subframes, n_channels, n_frames)
labels = c3d_ex['parameters']['POINT']['LABELS']['value']
analog_labels = c3d_ex['parameters']['ANALOG']['LABELS']['value']

# Step 3: Plot a marker trajectory (first marker)
m = 0
x, y, z = points[0, m, :], points[1, m, :], points[2, m, :]
plt.figure(); plt.plot(x, label='X'); plt.plot(y, label='Y'); plt.plot(z, label='Z')
plt.title(f'Marker trajectory: {labels[m]}'); plt.xlabel('Frame'); plt.ylabel('Position (mm)'); plt.legend(); plt.show()

# Step 4: Plot first analog channel (force plate, if available)
if analogs.size > 0:
    sig = analogs[0, 0, :]
    plt.figure(); plt.plot(sig); plt.title(f'Analog signal: {analog_labels[0] if len(analog_labels)>0 else "Ch0"}')
    plt.xlabel('Frame'); plt.ylabel('Value'); plt.show()

# Step 5: Toy cycle-normalized vertical GRF example (if label contains Fz)
if len(analog_labels) > 0 and any('Fz' in str(l) for l in analog_labels):
    # pick the first channel with 'Fz'
    j = next(i for i,l in enumerate(analog_labels) if 'Fz' in str(l))
    vGRF = analogs[0, j, :]
    stride = vGRF[:min(200, vGRF.shape[0])]
    stride_norm = np.linspace(0, 100, stride.shape[0])
    plt.figure(); plt.plot(stride_norm, stride)
    plt.title('Vertical GRF (one stride, normalized 0–100%)'); plt.xlabel('Gait Cycle (%)'); plt.ylabel('Force'); plt.show()
